In [26]:
import numpy as np
from sklearn.model_selection import train_test_split, train_test_split
import pandas as pd

from utils.data import extract_eeg_data, segment_data
from load_data import load_kfold_data, load_data
from utils.labels import get_pss_labels, get_stai_labels
from utils.valid_recs import get_valid_recs
from features import time_series_features, fractal_features, entropy_features,freq_band_features, hjorth_features, kymatio_wave_scattering
from classifiers import knn_classification, svm_classification

import utils.variables as v

In [27]:
def load_and_shape_data(data_type, label_type, features, kfold, new_ica = False):
    #Load data
    if kfold:
        train_data, test_data, train_labels, test_labels = load_kfold_data(data_type, label_type, epoched = False, binary = True)
    else:
        train_data, test_data, val_data, train_labels, test_labels, val_labels = load_data(data_type, label_type, epoched = True, binary = True)


    if features:
        #Reshape labels - ONLY FOR USING FEATURES
        train_labels = np.repeat(train_labels, repeats = 8, axis = 0).reshape((train_data.shape[0]*8,1))
        train_labels = train_labels.ravel()

        test_labels = np.repeat(test_labels,repeats = 8, axis = 0).reshape((test_data.shape[0]*8,1))
        test_labels = test_labels.ravel()

        print(train_labels.shape)
        print(test_labels.shape)

        #Extract features
        train_data = time_series_features(train_data, new_ica)
        test_data = time_series_features(test_data, new_ica)

        print(train_data.shape)
        print(test_data.shape)
    else:
        #Reshape data - ONLY FOR USING THE WHOLE DATASET, NOT FEATURES
        train_data = np.reshape(train_data, (train_data.shape[0]*train_data.shape[1], train_data.shape[2]))
        train_labels = np.repeat(train_labels, repeats = 8, axis = 1).reshape(-1,1)
        train_labels = train_labels.ravel()

        test_data = np.reshape(test_data, (test_data.shape[0]*test_data.shape[1],test_data.shape[2]))
        test_labels = np.repeat(test_labels, repeats = 8, axis = 1).reshape(-1,1)
        test_labels = test_labels.ravel()
    return train_data, test_data, train_labels, test_labels



In [44]:
data_type = 'raw'
label_type = 'pss'
features = True
kfold = True
feature = time_series_features


train_data, test_data, train_labels, test_labels = load_and_shape_data(data_type, label_type, features, kfold, feature)

---- Generating all recordings ----
All records generated

---- Filtering out invalid recordings ----
ERROR 1) Failed to read data for recording P006_S002_001
ERROR 1) Failed to read data for recording P006_S002_002
ERROR 1) Failed to read data for recording P028_S002_001
ERROR 1) Failed to read data for recording P028_S002_002

---- Returning valid recordings ----
['P001_S001_001', 'P001_S001_002', 'P001_S002_001', 'P001_S002_002', 'P002_S001_001', 'P002_S001_002', 'P002_S002_001', 'P002_S002_002', 'P003_S001_001', 'P003_S001_002', 'P003_S002_001', 'P003_S002_002', 'P004_S001_001', 'P004_S001_002', 'P004_S002_001', 'P004_S002_002', 'P005_S001_001', 'P005_S001_002', 'P005_S002_001', 'P005_S002_002', 'P006_S001_001', 'P006_S001_002', 'P007_S001_001', 'P007_S001_002', 'P007_S002_001', 'P007_S002_002', 'P008_S001_001', 'P008_S001_002', 'P008_S002_001', 'P008_S002_002', 'P009_S001_001', 'P009_S001_002', 'P009_S002_001', 'P009_S002_002', 'P010_S001_002', 'P010_S002_001', 'P010_S002_002', 'P

In [42]:
print(f'{data_type} data with {label_type} labels')
knn_classification(train_data, test_data, train_labels, test_labels)

raw data with pss labels
KNN:

 Confusion matrix:
[[58  6]
 [16 16]]
Accuracy, Sensitivity, Specificity:

[77.08 78.38 72.73]


In [43]:
print(f'{data_type} data with {label_type} labels')
svm_classification(train_data, test_data, train_labels, test_labels)


raw data with pss labels
SVM:

 Confusion matrix:
[[56  8]
 [14 18]]
Accuracy, Sensitivity, Specificity:
[77.08 80.   69.23]


In [31]:
data_type = 'init'

train_data, test_data, train_labels, test_labels = load_and_shape_data(data_type, label_type, features, kfold)

---- Generating all recordings ----
All records generated

---- Filtering out invalid recordings ----
ERROR 1) Failed to read data for recording P006_S002_001
ERROR 1) Failed to read data for recording P006_S002_002
ERROR 1) Failed to read data for recording P010_S001_001
ERROR 1) Failed to read data for recording P013_S001_001
ERROR 1) Failed to read data for recording P013_S001_002
ERROR 1) Failed to read data for recording P020_S001_001
ERROR 1) Failed to read data for recording P023_S002_002
ERROR 1) Failed to read data for recording P028_S001_001
ERROR 1) Failed to read data for recording P028_S001_002

---- Returning valid recordings ----
['P001_S001_001', 'P001_S001_002', 'P001_S002_001', 'P001_S002_002', 'P002_S001_001', 'P002_S001_002', 'P002_S002_001', 'P002_S002_002', 'P003_S001_001', 'P003_S001_002', 'P003_S002_001', 'P003_S002_002', 'P004_S001_001', 'P004_S001_002', 'P004_S002_001', 'P004_S002_002', 'P005_S001_001', 'P005_S001_002', 'P005_S002_001', 'P005_S002_002', 'P006_

In [32]:
print(f'{data_type} data with {label_type} labels')
knn_classification(train_data, test_data, train_labels, test_labels)

init data with pss labels
KNN:

 Confusion matrix:
[[62  2]
 [29  3]]
Accuracy, Sensitivity, Specificity:

[67.71 68.13 60.  ]


In [33]:
print(f'{data_type} data with {label_type} labels')
svm_classification(train_data, test_data, train_labels, test_labels)

init data with pss labels
SVM:

 Confusion matrix:
[[63  1]
 [28  4]]
Accuracy, Sensitivity, Specificity:
[69.79 69.23 80.  ]


In [34]:
data_type = 'ica'

train_data, test_data, train_labels, test_labels = load_and_shape_data(data_type, label_type, features, kfold)

---- Generating all recordings ----
All records generated

---- Filtering out invalid recordings ----
ERROR 1) Failed to read data for recording P006_S002_001
ERROR 1) Failed to read data for recording P006_S002_002
ERROR 1) Failed to read data for recording P028_S001_001
ERROR 1) Failed to read data for recording P028_S001_002

---- Returning valid recordings ----
['P001_S001_001', 'P001_S001_002', 'P001_S002_001', 'P001_S002_002', 'P002_S001_001', 'P002_S001_002', 'P002_S002_001', 'P002_S002_002', 'P003_S001_001', 'P003_S001_002', 'P003_S002_001', 'P003_S002_002', 'P004_S001_001', 'P004_S001_002', 'P004_S002_001', 'P004_S002_002', 'P005_S001_001', 'P005_S001_002', 'P005_S002_001', 'P005_S002_002', 'P006_S001_001', 'P006_S001_002', 'P007_S001_001', 'P007_S001_002', 'P007_S002_001', 'P007_S002_002', 'P008_S001_001', 'P008_S001_002', 'P008_S002_001', 'P008_S002_002', 'P009_S001_001', 'P009_S001_002', 'P009_S002_001', 'P009_S002_002', 'P010_S001_002', 'P010_S002_001', 'P010_S002_002', 'P

In [35]:
print(f'{data_type} data with {label_type} labels')
knn_classification(train_data, test_data, train_labels, test_labels)

ica data with pss labels
KNN:

 Confusion matrix:
[[55  9]
 [30  2]]
Accuracy, Sensitivity, Specificity:

[59.38 64.71 18.18]


In [36]:
print(f'{data_type} data with {label_type} labels')
svm_classification(train_data, test_data, train_labels, test_labels)

ica data with pss labels
SVM:

 Confusion matrix:
[[61  3]
 [28  4]]
Accuracy, Sensitivity, Specificity:
[67.71 68.54 57.14]


In [37]:
data_type = 'new_ica'

train_data, test_data, train_labels, test_labels = load_and_shape_data(data_type, label_type, features, kfold, new_ica = False)

---- Generating all recordings ----
All records generated

---- Filtering out invalid recordings ----
ERROR 1) Failed to read data for recording P006_S002_001
ERROR 1) Failed to read data for recording P006_S002_002
ERROR 1) Failed to read data for recording P010_S001_001
ERROR 1) Failed to read data for recording P013_S001_001
ERROR 1) Failed to read data for recording P013_S001_002
ERROR 1) Failed to read data for recording P020_S001_001
ERROR 1) Failed to read data for recording P023_S002_002
ERROR 1) Failed to read data for recording P028_S001_001
ERROR 1) Failed to read data for recording P028_S001_002

---- Returning valid recordings ----
['P001_S001_001', 'P001_S001_002', 'P001_S002_001', 'P001_S002_002', 'P002_S001_001', 'P002_S001_002', 'P002_S002_001', 'P002_S002_002', 'P003_S001_001', 'P003_S001_002', 'P003_S002_001', 'P003_S002_002', 'P004_S001_001', 'P004_S001_002', 'P004_S002_001', 'P004_S002_002', 'P005_S001_001', 'P005_S001_002', 'P005_S002_001', 'P005_S002_002', 'P006_

In [38]:
print(f'{data_type} data with {label_type} labels')
knn_classification(train_data, test_data, train_labels, test_labels)

new_ica data with pss labels
KNN:

 Confusion matrix:
[[64  0]
 [32  0]]
Null error in specificity
Accuracy, Sensitivity, Specificity:

[66.67 66.67  0.  ]


In [39]:
print(f'{data_type} data with {label_type} labels')
svm_classification(train_data, test_data, train_labels, test_labels)

new_ica data with pss labels
SVM:

 Confusion matrix:
[[64  0]
 [32  0]]
Null error in specificity
Accuracy, Sensitivity, Specificity:
[66.67 66.67  0.  ]
